# bob4

### STATUS

#### create_dataset_fcn8
- Basically able to return a fuzzy version of the input
- Not sure how much more I can tweak the model to reach the next time stamp

#### create_dataset_diff
- Currently doing **validation data** on both training and evaluation!!
- Trying to overfit and show validation function works, too

#### LSTM
- I think I need to add the RNN portion to get it to learn where to go from the start

### TODO
- Figure out memory overflow
 - 2018-06-04 07:48:12.842444: W tensorflow/core/common_runtime/bfc_allocator.cc:219] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.34GiB. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.

In [1]:
import os
import numpy as np
import tensorflow as wtf
import tensorflow.contrib.rnn as rnn
import data_utils as du

/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
BATCH_SIZE = 4
EPOCHS = 1

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

In [3]:
MODEL_TYPE = 'lstm' # choices: model (basic non-diff), diff, lstm
OVERFIT = True

MODEL_NAME = MODEL_TYPE + '001'

In [4]:
def encoder_fn(inputs):
    conv1_1 = wtf.layers.conv2d(inputs,
                                filters=64,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv1_1')
    conv1_2 = wtf.layers.conv2d(conv1_1,
                                filters=64,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv1_2')
    pool1 = wtf.layers.max_pooling2d(conv1_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool1')
        
    conv2_1 = wtf.layers.conv2d(pool1,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv2_1')
    conv2_2 = wtf.layers.conv2d(conv2_1,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv2_2')
    pool2 = wtf.layers.max_pooling2d(conv2_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool2')
        
    conv3_1 = wtf.layers.conv2d(pool2,
                                filters=1024,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_1')
    conv3_2 = wtf.layers.conv2d(conv3_1,
                                filters=1024,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_2')
    pool3 = wtf.layers.max_pooling2d(conv3_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool3')
        
    conv4_1 = wtf.layers.conv2d(pool3,
                                filters=2048,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_1')
    conv4_2 = wtf.layers.conv2d(conv4_1,
                                filters=1,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_2')
    conv4_3 = wtf.layers.conv2d(conv4_2,
                                filters=1,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_3')
        
    dout = wtf.layers.dropout(conv4_3,
                              rate=0.0)
    
    outputs = dout
    return outputs, pool1, pool2

In [5]:
def decoder_fn(inputs, pool1, pool2):
    deconv1 = wtf.layers.conv2d_transpose(inputs,
                                          filters=pool2.get_shape().as_list()[-1],
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    deconv1_skip = wtf.add(deconv1, pool2)
    
    deconv2 = wtf.layers.conv2d_transpose(deconv1_skip,
                                          filters=pool1.get_shape().as_list()[-1],
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    deconv2_skip = wtf.add(deconv2, pool1)
    
    deconv3 = wtf.layers.conv2d_transpose(deconv2_skip,
                                          filters=1,
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    
    outputs = deconv3
    return outputs

In [ ]:
# Functioning t -> t+1 model, but doesn't do a great job at predicting

def model_fn(features,
             labels,
             mode,
             params):
    with wtf.device('/device:GPU:0'):
        inputs = features
        
        dout, pool1, pool2 = encoder_fn(inputs)
        
        deconv3 = decoder_fn(dout, pool1, pool2)
        
#         outputs = wtf.add(deconv2, inputs)
        outputs = deconv3

#         wtf.summary.image('input', inputs, max_outputs=200)
#         wtf.summary.image('outputs', outputs, max_outputs=200)
#         wtf.summary.image('labels', labels, max_outputs=200)
        
        # Adds concatenated inputs and outputs to be displayed in TensorBoard
        disp = wtf.concat([inputs, labels, outputs], 1)
        wtf.summary.image('concat', disp, max_outputs=200)
    
        predictions = {'results': outputs}
    
        if mode == wtf.estimator.ModeKeys.PREDICT:
            export_outputs = {
                'predictions': wtf.estimator.export.PredictOutput(outputs)
            }
            return wtf.estimator.EstimatorSpec(
                mode=mode,
                predictions=outputs
            )
    
        # loss = wtf.losses.mean_squared_error(labels, outputs)
        loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[1,2,3])
        loss = wtf.reduce_mean(loss)
        # Idea: add max of outputs when using diff data
        # Thought is this will really penalize when the diff has bright spots vs when its mostly dark?
#         loss = loss + 10*wtf.reduce_max(outputs)
        
        global_step = wtf.train.get_global_step()
        lr = wtf.train.exponential_decay(params.learning_rate, global_step, 1000, 0.96, staircase=True)
        
        wtf.summary.scalar('learning_rate', lr)
        
        optimizer = wtf.train.AdamOptimizer(
            learning_rate=lr,
            beta1=params.beta1,
            beta2=params.beta2
        )
    
        update_ops = wtf.get_collection(wtf.GraphKeys.UPDATE_OPS)
        with wtf.control_dependencies(update_ops):
            train_op = optimizer.minimize(
                loss=loss, global_step=wtf.train.get_global_step())
    
        eval_metric_ops = {
            "rmse": wtf.metrics.root_mean_squared_error(labels, outputs),
            "mae": wtf.metrics.mean_absolute_error(labels, outputs),
            "mse": wtf.metrics.mean_squared_error(labels, outputs)
        }
        
        eval_summary_hook = wtf.train.SummarySaverHook(
                                save_steps=1,
                                output_dir= "summary/eval/" + MODEL_NAME,
                                scaffold=wtf.train.Scaffold(summary_op=wtf.summary.merge_all()))
        
        evaluation_hooks=[]
        evaluation_hooks.append(eval_summary_hook)
    
        estimator_spec = wtf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops,
            evaluation_hooks=evaluation_hooks
        )
        
        return estimator_spec

In [6]:
# Adding the recurrence

def model_fn(features,
             labels,
             mode,
             params):
    with wtf.device('/device:GPU:0'):
        inputs = features
        print(inputs)
        # Is there a better way of doing this?? Haha. # # # # # #
        inputs_tr = wtf.transpose(inputs, [2, 3, 4, 1, 0])
        inputs_re = wtf.reshape(inputs_tr, [192, 224, 1, -1])
        inputs_NHWC = wtf.transpose(inputs_re, [3, 0, 1, 2])
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
        dout, pool1, pool2 = encoder_fn(inputs_NHWC) # dout (17*N, 24, 28, 1)
        dout_tr = wtf.transpose(dout, [1, 2, 3, 0])
        dout_re = wtf.reshape(dout_tr, [24*28*1, 17, -1])
        dout_NTC = wtf.transpose(dout_re, [2, 1, 0])
        lstm_cell = rnn.LSTMCell(num_units=24*28*1,
                                 forget_bias=1.0,
                                 reuse=None,
                                 name=None)
        rnn_outputs, _ = wtf.nn.dynamic_rnn(cell=lstm_cell,
                                            inputs=dout_NTC,
                                            dtype=wtf.float32)
#         print(rnn_outputs) # I'm not sure what shape the rnn outputs
        rnn_tr = wtf.transpose(rnn_outputs, [2, 1, 0])
        rnn_re = wtf.reshape(rnn_tr, [24, 28, 1, -1])
        rnn_NHWC = wtf.transpose(rnn_re, [3, 0, 1, 2])
        deconv3 = decoder_fn(rnn_NHWC, pool1, pool2)
#         outputs = wtf.add(deconv2, inputs)
        dc3_tr = wtf.transpose(deconv3, [1, 2, 3, 0])
        dc3_re = wtf.reshape(dc3_tr, [192, 224, 1, 17, -1])
        outputs = wtf.transpose(dc3_re, [4, 3, 0, 1, 2])
        print(outputs)

#         wtf.summary.image('input', inputs, max_outputs=200)
#         wtf.summary.image('outputs', outputs, max_outputs=200)
#         wtf.summary.image('labels', labels, max_outputs=200)
        
        # Adds concatenated inputs and outputs to be displayed in TensorBoard
        disp = wtf.concat([inputs, labels, outputs], 1)
        wtf.summary.image('concat', disp, max_outputs=200)
    
        predictions = {'results': outputs}
    
        if mode == wtf.estimator.ModeKeys.PREDICT:
            export_outputs = {
                'predictions': wtf.estimator.export.PredictOutput(outputs)
            }
            return wtf.estimator.EstimatorSpec(
                mode=mode,
                predictions=outputs
            )
    
        # loss = wtf.losses.mean_squared_error(labels, outputs)
        loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[1,2,3])
        loss = wtf.reduce_mean(loss)
        
        global_step = wtf.train.get_global_step()
        lr = wtf.train.exponential_decay(params.learning_rate, global_step, 1000, 0.96, staircase=True)
        
        wtf.summary.scalar('learning_rate', lr)
        
        optimizer = wtf.train.AdamOptimizer(
            learning_rate=lr,
            beta1=params.beta1,
            beta2=params.beta2
        )
    
        update_ops = wtf.get_collection(wtf.GraphKeys.UPDATE_OPS)
        with wtf.control_dependencies(update_ops):
            train_op = optimizer.minimize(
                loss=loss, global_step=wtf.train.get_global_step())
    
        eval_metric_ops = {
            "rmse": wtf.metrics.root_mean_squared_error(labels, outputs),
            "mae": wtf.metrics.mean_absolute_error(labels, outputs),
            "mse": wtf.metrics.mean_squared_error(labels, outputs)
        }
        
        eval_summary_hook = wtf.train.SummarySaverHook(
                                save_steps=1,
                                output_dir= "summary/eval/" + MODEL_NAME,
                                scaffold=wtf.train.Scaffold(summary_op=wtf.summary.merge_all()))
        
        evaluation_hooks=[]
        evaluation_hooks.append(eval_summary_hook)
    
        estimator_spec = wtf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops,
            evaluation_hooks=evaluation_hooks
        )
        
        return estimator_spec

In [7]:
def main(_):
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    
    wtf.logging.set_verbosity(wtf.logging.INFO)

    hparams = wtf.contrib.training.HParams(
        #num_epochs=1,
        #batch_size = 80,
        #forget_bias=1.0,
        learning_rate=0.00001,
        beta1=0.9,
        beta2=0.999
    )
    
    session_config = wtf.ConfigProto()
    session_config.gpu_options.allow_growth = True
    session_config.allow_soft_placement = True
    
    run_config = wtf.estimator.RunConfig(
        log_step_count_steps=10,
        save_summary_steps=10,
        tf_random_seed=19830610,
        model_dir=os.path.join('summary', 'train', MODEL_NAME),
        session_config=session_config
    )
    
    estimator = wtf.estimator.Estimator(
        model_fn=model_fn, params=hparams, config=run_config)
    
    if MODEL_TYPE=='lstm':
        if OVERFIT:
            train_dataset = du.create_dataset('val', val_data)
        else:
            train_dataset = du.create_dataset('train', train_data)
    elif MODEL_TYPE=='model':
        if OVERFIT:
            train_dataset = du.create_dataset_fcn8('val', val_data)
        else:
            train_dataset = du.create_dataset_fcn8('train', train_data)
    elif MODEL_TYPE=='diff':
        if OVERFIT:
            train_dataset = du.create_dataset_diff('val', val_data)
        else:
            train_dataset = du.create_dataset_diff('train', train_data)
    train_dataset = train_dataset.shuffle(2000)
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.repeat(EPOCHS)
    
    if MODEL_TYPE=='lstm':
        val_dataset = du.create_dataset('val', val_data)
    elif MODEL_TYPE=='model':
        val_dataset = du.create_dataset_fcn8('val', val_data)
    elif MODEL_TYPE=='diff':
        val_dataset = du.create_dataset_diff('val', val_data)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.repeat(1)
    
    def train_input_fn():
        train_iterator = train_dataset.make_one_shot_iterator()
        features, labels = train_iterator.get_next()
        return features, labels
    
    def val_input_fn():
        val_iterator = val_dataset.make_one_shot_iterator()
        features, labels = val_iterator.get_next()
        return features, labels
    
    estimator.train(input_fn=train_input_fn, max_steps=None)
    estimator.evaluate(input_fn=val_input_fn)

In [8]:
if __name__ == '__main__':
    wtf.app.run()

INFO:tensorflow:Using config: {'_model_dir': 'summary/train/lstm001', '_tf_random_seed': 19830610, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f07d5e06ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Tensor("IteratorGetNext:0", shape=(?, 17, 192, 224, 1), dtype=float32, device=/device:CPU:0)
Tensor("transpose_7:0", shape=(?, 17, 192, 224, 1), dtype=float32, device=/device:GPU:0)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
IN

InvalidArgumentError: Tensor must be 4-D with last dim 1, 3, or 4, not [4,51,192,224,1]
	 [[Node: concat_1 = ImageSummary[T=DT_FLOAT, bad_color=Tensor<type: uint8 shape: [4] values: 255 0 0...>, max_images=200, _device="/job:localhost/replica:0/task:0/device:CPU:0"](concat_1/tag, concat/_391)]]

Caused by op 'concat_1', defined at:
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    s